In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models

# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([ transforms.Resize((224,224)), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize( (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) ) ])
# Define data transforms and load CIFAR-100 dataset

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)



100%|██████████| 169001437/169001437 [00:01<00:00, 105635254.50it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
# Define the VGG16 model with custom classifier for 100 classes
model = models.vgg16(pretrained=True)  # Load pre-trained weights
model.classifier[6] = nn.Linear(4096, 100)  # Change the output layer for 100 classes

# Send the model to the selected device
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
for epoch in range(10):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # Print every 200 batches
            print(f"Epoch[ {epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}")
            running_loss = 0.0

print("Finished Training")

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 160MB/s]


Epoch[ 1, 200] loss: 3.095
Epoch[ 1, 400] loss: 1.662
Epoch[ 1, 600] loss: 1.448
Epoch[ 2, 200] loss: 1.152
Epoch[ 2, 400] loss: 1.102
Epoch[ 2, 600] loss: 1.080
Epoch[ 3, 200] loss: 0.893
Epoch[ 3, 400] loss: 0.878
Epoch[ 3, 600] loss: 0.859
Epoch[ 4, 200] loss: 0.717
Epoch[ 4, 400] loss: 0.745
Epoch[ 4, 600] loss: 0.756
Epoch[ 5, 200] loss: 0.619
Epoch[ 5, 400] loss: 0.624
Epoch[ 5, 600] loss: 0.654
Epoch[ 6, 200] loss: 0.523
Epoch[ 6, 400] loss: 0.548
Epoch[ 6, 600] loss: 0.545
Epoch[ 7, 200] loss: 0.444
Epoch[ 7, 400] loss: 0.470
Epoch[ 7, 600] loss: 0.483
Epoch[ 8, 200] loss: 0.388
Epoch[ 8, 400] loss: 0.397
Epoch[ 8, 600] loss: 0.400
Epoch[ 9, 200] loss: 0.329
Epoch[ 9, 400] loss: 0.339
Epoch[ 9, 600] loss: 0.349
Epoch[ 10, 200] loss: 0.298
Epoch[ 10, 400] loss: 0.302
Epoch[ 10, 600] loss: 0.311
Finished Training
Accuracy on the test set: 74.14%


In [3]:
# Evaluate the model on the training set
train_correct = 0
train_total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

print(f"Accuracy on the training set: {100 * train_correct / train_total:.2f}%")


Accuracy on the training set: 91.98%
